In [1]:
import math
import tensorflow as tf
import numpy as np
import matplotlib
import os
from tensorflow.contrib import rnn

tf.reset_default_graph()
tf.set_random_seed(777)

if "DISPLAY" not in os.environ:
    matplotlib.use('TkAgg')

import matplotlib.pyplot as plt

def MinMaxScaler(data):
    numerator = data - np.min(data,0)
    denominator = np.max(data,0)-np.min(data,0)
    return numerator / (denominator+1e-7)

xy = np.genfromtxt('/Users/yeseo/Desktop/City_Counted_TaxiMach_Link_Dataset_Full_201501 - 12.txt',delimiter = ',',dtype = None)
xy_with_noise = np.genfromtxt('/Users/yeseo/Desktop/2015eliminated_1.txt',delimiter = ',',dtype = None)

#data_preprocessing
xy= xy[:,:27]
a = xy[:,:2]
b = xy[:,2:]
b= MinMaxScaler(b)
xy = np.hstack((a,b))

xy_with_noise = xy_with_noise[:,:27]
a_with_noise = xy_with_noise[:,:2]
b_with_noise = xy_with_noise[:,2:]
b_with_noise = MinMaxScaler(b_with_noise)
xy_with_noise = np.hstack((a_with_noise,b_with_noise))


#parameters
seq_length =6
data_dim =27
hidden_dim = 54
output_dim = 25
learning_rate = 0.01
iterations = 4000

train_size = int(len(xy)*0.7)
validation_size = int(len(xy)*0.2)

#divide data set to train,validation and test set
train_set = xy[:train_size]
validation_set = xy[train_size:train_size+validation_size]
test_set = xy[train_size+validation_size:]

train_set_with_noise = xy_with_noise[:train_size]
validation_set_with_noise = xy_with_noise[train_size:train_size+validation_size]
test_set_with_noise = xy_with_noise[train_size+validation_size:]

# build data set for rnn
def build_dataset(time_series, seq_length):
    dataX = []
    dataY = []
    for i in range (0,len(time_series)-seq_length):
        _x = time_series[i:i + seq_length, :]
        _y = time_series[i+seq_length,2:]
     
        dataX.append(_x)
        dataY.append(_y)
    return np.array(dataX), np.array(dataY)

#train_set, test_set 만들기
trainX, trainY = build_dataset(train_set,seq_length)
validationX, validationY = build_dataset(validation_set,seq_length)
testX,testY = build_dataset(test_set, seq_length)

trainX_with_noise, trainY_with_noise = build_dataset(train_set_with_noise,seq_length)
validationX_with_noise, validationY_with_noise = build_dataset(validation_set_with_noise,seq_length)
testX_with_noise,testY_with_noise = build_dataset(test_set_with_noise, seq_length)


X1 = tf.placeholder(tf.float32,[None, seq_length,data_dim])
Y1 = tf.placeholder(tf.float32,[None,25])

X2 = tf.placeholder(tf.float32,[None, seq_length,data_dim])
Y2 = tf.placeholder(tf.float32,[None,25])

#LSTM CELL만들기

with tf.variable_scope("rnn1"):
    cell1 = tf.contrib.rnn.BasicLSTMCell(num_units=hidden_dim, state_is_tuple = True, activation=tf.tanh)
    outputs1,_states1 = tf.nn.dynamic_rnn(cell1,X1,dtype = tf.float32)
    Y_pred = tf.contrib.layers.fully_connected(outputs1[:,-1], output_dim,activation_fn = None)

    loss1 =tf.reduce_sum(tf.square(Y_pred-Y1))
    train1 = tf.train.AdamOptimizer(learning_rate = learning_rate).minimize(loss1)

with tf.variable_scope("rnn2"):
    cell2 = tf.contrib.rnn.BasicLSTMCell(num_units=hidden_dim, state_is_tuple = True, activation=tf.tanh)
    outputs2,_states2 = tf.nn.dynamic_rnn(cell2, X2, dtype = tf.float32)
    Y_pred_with_noise = tf.contrib.layers.fully_connected(outputs2[:,-1], output_dim,activation_fn = None)

    loss2 =tf.reduce_mean(tf.square(Y_pred_with_noise-Y2))
    train2 = tf.train.AdamOptimizer(learning_rate = learning_rate).minimize(loss2)


#RMSE 측정
targets = tf.placeholder(tf.float32,[None,25])
predictions = tf.placeholder(tf.float32,[None,25])
rmse = tf.sqrt(tf.reduce_mean(tf.square(targets-predictions)))

x1 = np.array([1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25])
x2 = x1+0.3
x3 = x2+0.3
loss_for_graph = np.zeros(iterations)
x4 = np.array(range(0,iterations))
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())

    for i in range(iterations):
        _, step_loss1 = sess.run([train1,loss1],feed_dict={X1:trainX, Y1:trainY})
        print("[step: {}] loss: {}".format(i,step_loss1))
        loss_for_graph[i] = step_loss1
        _, step_loss2 = sess.run([train2,loss2],feed_dict={X2:trainX_with_noise, Y2:trainY_with_noise})
        print("[step: {}] loss: {}".format(i,step_loss2))
        
    validation_predict = sess.run(Y_pred, feed_dict = {X1:validationX})
    validation_predict_with_noise = sess.run(Y_pred_with_noise, feed_dict = {X2:validationX_with_noise})
    test_predict = sess.run(Y_pred, feed_dict = {X1:testX})
    
    rmse_val = sess.run(rmse, feed_dict={targets: validationY,predictions: validation_predict})
    print("RMSE: {}".format(rmse_val))
   # print("pred: {}".format(test_predict[-1,:]))
    #print("real: {}".format(testY[-1,:]))
    #print("noise: {}".format(eliminate_noise_pred[-1,:]))
    
#    plt.bar(x1,test_predict[-1,:],label = 'predict',color ='b',width = 0.1)
  #  plt.bar(x2,testY[-1,:],label = 'real',color ='g',width = 0.1)
    #plt.bar(x3,eliminate_noise_pred[-1,:],label = 'noise',color ='g',width = 0.1)
    plt.plot(x4,loss_for_graph)
    #plt.legend()
    plt.show()




Instructions for updating:
This class is deprecated, please use tf.nn.rnn_cell.LSTMCell, which supports all the feature this cell currently has. Please replace the existing code with tf.nn.rnn_cell.LSTMCell(name='basic_lstm_cell').
[step: 0] loss: 23487.666015625
[step: 0] loss: 0.5357515811920166
[step: 1] loss: 10543.6708984375
[step: 1] loss: 0.18657314777374268
[step: 2] loss: 5960.580078125
[step: 2] loss: 0.08881929516792297
[step: 3] loss: 4311.20556640625
[step: 3] loss: 0.06655864417552948
[step: 4] loss: 3199.855224609375
[step: 4] loss: 0.05936209112405777
[step: 5] loss: 2464.709228515625
[step: 5] loss: 0.04926927015185356
[step: 6] loss: 2108.736572265625
[step: 6] loss: 0.04052507504820824
[step: 7] loss: 1985.2215576171875
[step: 7] loss: 0.03440467268228531
[step: 8] loss: 1888.045166015625
[step: 8] loss: 0.03055683523416519
[step: 9] loss: 1785.1380615234375
[step: 9] loss: 0.02783109061419964
[step: 10] loss: 1686.0806884765625
[step: 10] loss: 0.025544999167323112


[step: 108] loss: 0.008242701180279255
[step: 109] loss: 173.05545043945312
[step: 109] loss: 0.008228112943470478
[step: 110] loss: 172.146728515625
[step: 110] loss: 0.008263071067631245
[step: 111] loss: 171.40940856933594
[step: 111] loss: 0.008104214444756508
[step: 112] loss: 170.7532958984375
[step: 112] loss: 0.008221983909606934
[step: 113] loss: 170.73123168945312
[step: 113] loss: 0.008058041334152222
[step: 114] loss: 171.35189819335938
[step: 114] loss: 0.008149145171046257
[step: 115] loss: 173.88233947753906
[step: 115] loss: 0.008068486116826534
[step: 116] loss: 178.29710388183594
[step: 116] loss: 0.008042276836931705
[step: 117] loss: 187.82443237304688
[step: 117] loss: 0.008078396320343018
[step: 118] loss: 190.8101806640625
[step: 118] loss: 0.007970314472913742
[step: 119] loss: 186.78309631347656
[step: 119] loss: 0.008027669042348862
[step: 120] loss: 168.62725830078125
[step: 120] loss: 0.007959937676787376
[step: 121] loss: 163.61459350585938
[step: 121] loss

[step: 218] loss: 0.007323427125811577
[step: 219] loss: 120.06436920166016
[step: 219] loss: 0.007329947780817747
[step: 220] loss: 119.79732513427734
[step: 220] loss: 0.007321934215724468
[step: 221] loss: 119.503662109375
[step: 221] loss: 0.007304364815354347
[step: 222] loss: 119.136474609375
[step: 222] loss: 0.0072929104790091515
[step: 223] loss: 118.75481414794922
[step: 223] loss: 0.007292208261787891
[step: 224] loss: 118.41272735595703
[step: 224] loss: 0.007298001553863287
[step: 225] loss: 118.05220031738281
[step: 225] loss: 0.007303834427148104
[step: 226] loss: 117.70454406738281
[step: 226] loss: 0.007303827442228794
[step: 227] loss: 117.39717102050781
[step: 227] loss: 0.007300115190446377
[step: 228] loss: 117.07806396484375
[step: 228] loss: 0.00729186600074172
[step: 229] loss: 116.75946807861328
[step: 229] loss: 0.00728365033864975
[step: 230] loss: 116.46839904785156
[step: 230] loss: 0.007275510113686323
[step: 231] loss: 116.19541931152344
[step: 231] loss:

[step: 327] loss: 0.0070769754238426685
[step: 328] loss: 93.13146209716797
[step: 328] loss: 0.007110888604074717
[step: 329] loss: 94.27710723876953
[step: 329] loss: 0.007157403975725174
[step: 330] loss: 98.3546142578125
[step: 330] loss: 0.007117704953998327
[step: 331] loss: 101.55908203125
[step: 331] loss: 0.007067367900162935
[step: 332] loss: 98.72811889648438
[step: 332] loss: 0.007084497716277838
[step: 333] loss: 94.16868591308594
[step: 333] loss: 0.0071160560473799706
[step: 334] loss: 91.43917846679688
[step: 334] loss: 0.007100773509591818
[step: 335] loss: 92.56117248535156
[step: 335] loss: 0.007062881253659725
[step: 336] loss: 95.11174011230469
[step: 336] loss: 0.007062991615384817
[step: 337] loss: 95.54942321777344
[step: 337] loss: 0.007088192272931337
[step: 338] loss: 93.99620056152344
[step: 338] loss: 0.007087424863129854
[step: 339] loss: 91.16932678222656
[step: 339] loss: 0.007063454017043114
[step: 340] loss: 90.06597900390625
[step: 340] loss: 0.007048

[step: 437] loss: 83.72055053710938
[step: 437] loss: 0.00692030368372798
[step: 438] loss: 99.2134017944336
[step: 438] loss: 0.006947655696421862
[step: 439] loss: 84.15455627441406
[step: 439] loss: 0.007019482087343931
[step: 440] loss: 76.675048828125
[step: 440] loss: 0.007019786164164543
[step: 441] loss: 83.74314880371094
[step: 441] loss: 0.006959341932088137
[step: 442] loss: 85.44319152832031
[step: 442] loss: 0.006911184173077345
[step: 443] loss: 79.48114013671875
[step: 443] loss: 0.006930484902113676
[step: 444] loss: 75.67027282714844
[step: 444] loss: 0.0069717164151370525
[step: 445] loss: 84.21612548828125
[step: 445] loss: 0.006965982262045145
[step: 446] loss: 82.9920883178711
[step: 446] loss: 0.006926306523382664
[step: 447] loss: 75.11137390136719
[step: 447] loss: 0.006902365945279598
[step: 448] loss: 80.16683959960938
[step: 448] loss: 0.00691800843924284
[step: 449] loss: 81.05003356933594
[step: 449] loss: 0.006941545754671097
[step: 450] loss: 75.991348266

...

[step: 3649] loss: 0.005670422688126564
[step: 3650] loss: 19.447696685791016
[step: 3650] loss: 0.00569453090429306
[step: 3651] loss: 18.08542251586914
[step: 3651] loss: 0.0057113440707325935
[step: 3652] loss: 16.74899673461914
[step: 3652] loss: 0.005691110156476498
[step: 3653] loss: 16.773517608642578
[step: 3653] loss: 0.005656072404235601
[step: 3654] loss: 16.792884826660156
[step: 3654] loss: 0.005604610312730074
[step: 3655] loss: 15.880636215209961
[step: 3655] loss: 0.005579749122262001
[step: 3656] loss: 15.15652084350586
[step: 3656] loss: 0.005588062573224306
[step: 3657] loss: 15.386617660522461
[step: 3657] loss: 0.0056173717603087425
[step: 3658] loss: 15.713668823242188
[step: 3658] loss: 0.005644959397614002
[step: 3659] loss: 15.548325538635254
[step: 3659] loss: 0.0056258756667375565
[step: 3660] loss: 15.188520431518555
[step: 3660] loss: 0.00558497104793787
[step: 3661] loss: 15.407510757446289
[step: 3661] loss: 0.005547625944018364
[step: 3662] loss: 15.7698

[step: 3755] loss: 0.005538650788366795
[step: 3756] loss: 15.66850471496582
[step: 3756] loss: 0.005524809472262859
[step: 3757] loss: 15.203057289123535
[step: 3757] loss: 0.005510491319000721
[step: 3758] loss: 15.719350814819336
[step: 3758] loss: 0.005508920177817345
[step: 3759] loss: 15.196540832519531
[step: 3759] loss: 0.005516176111996174
[step: 3760] loss: 15.167658805847168
[step: 3760] loss: 0.005523924250155687
[step: 3761] loss: 15.356773376464844
[step: 3761] loss: 0.005534266121685505
[step: 3762] loss: 14.772713661193848
[step: 3762] loss: 0.005522832740098238
[step: 3763] loss: 15.105670928955078
[step: 3763] loss: 0.005523110739886761
[step: 3764] loss: 15.234699249267578
[step: 3764] loss: 0.005529994610697031
[step: 3765] loss: 14.954063415527344
[step: 3765] loss: 0.005540015641599894
[step: 3766] loss: 15.234687805175781
[step: 3766] loss: 0.005545578896999359
[step: 3767] loss: 14.948566436767578
[step: 3767] loss: 0.005551168229430914
[step: 3768] loss: 14.701

[step: 3861] loss: 0.005500744096934795
[step: 3862] loss: 16.002086639404297
[step: 3862] loss: 0.00550256809219718
[step: 3863] loss: 15.344406127929688
[step: 3863] loss: 0.005528823472559452
[step: 3864] loss: 14.608285903930664
[step: 3864] loss: 0.005518642719835043
[step: 3865] loss: 14.282346725463867
[step: 3865] loss: 0.005507588852196932
[step: 3866] loss: 14.584135055541992
[step: 3866] loss: 0.005527373403310776
[step: 3867] loss: 15.05991268157959
[step: 3867] loss: 0.005534475669264793
[step: 3868] loss: 15.138580322265625
[step: 3868] loss: 0.005518215708434582
[step: 3869] loss: 14.831401824951172
[step: 3869] loss: 0.005519455298781395
[step: 3870] loss: 14.463781356811523
[step: 3870] loss: 0.005517842713743448
[step: 3871] loss: 14.394832611083984
[step: 3871] loss: 0.005505444947630167
[step: 3872] loss: 14.628190994262695
[step: 3872] loss: 0.005486515816301107
[step: 3873] loss: 14.806992530822754
[step: 3873] loss: 0.00548947649076581
[step: 3874] loss: 14.80908

[step: 3967] loss: 0.005574430804699659
[step: 3968] loss: 13.770173072814941
[step: 3968] loss: 0.00555117754265666
[step: 3969] loss: 13.769681930541992
[step: 3969] loss: 0.005518828518688679
[step: 3970] loss: 13.863072395324707
[step: 3970] loss: 0.005494656041264534
[step: 3971] loss: 13.780372619628906
[step: 3971] loss: 0.005494119133800268
[step: 3972] loss: 13.568085670471191
[step: 3972] loss: 0.005500619765371084
[step: 3973] loss: 13.524681091308594
[step: 3973] loss: 0.00549348583444953
[step: 3974] loss: 13.615342140197754
[step: 3974] loss: 0.0054750884883105755
[step: 3975] loss: 13.642311096191406
[step: 3975] loss: 0.0054648215882480145
[step: 3976] loss: 13.567206382751465
[step: 3976] loss: 0.0054694609716534615
[step: 3977] loss: 13.51565933227539
[step: 3977] loss: 0.005482683423906565
[step: 3978] loss: 13.568077087402344
[step: 3978] loss: 0.005494528915733099
[step: 3979] loss: 13.645122528076172
[step: 3979] loss: 0.005501844920217991
[step: 3980] loss: 13.65

InvalidArgumentError: Incompatible shapes: [800,25] vs. [398,25]
	 [[node sub (defined at <ipython-input-1-6d038b9841fd>:108)  = Sub[T=DT_FLOAT, _device="/job:localhost/replica:0/task:0/device:CPU:0"](_arg_Placeholder_4_0_0, _arg_Placeholder_5_0_1)]]

Caused by op 'sub', defined at:
  File "/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel/kernelapp.py", line